<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>


<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use scikit-learn to predict hand-written digits</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/raw/master/scikit-learn/hand-written-digits-recognition/images/numbers_banner-04.png" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to work with [watson-machine-learning-client](https://pypi.python.org/pypi/watson-machine-learning-client) library available in PyPI repository. This notebook introduces commands for getting data and for basic data exploration, pipeline creation, model training and evaluation, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5, scikit-learn, watson-machine-learning-client package.

You will use a toy dataset available in scikit-learn, **sklearn.datasets.load_digits**, which contains hand-written digits images. Use the toy dataset to recognize hand-written digits.

## Learning goals

The learning goals of this notebook are:

-  Load a sample dataset from scikit-learn.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create Scikit-learn machine learning pipeline.
-  Train and evaluate a model.
-  Persist a model in Watson Machine Learning repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1.	[Set up the environment](#setup)
2.	[Load and explore data](#load)
3.	[Create scikit-learn model](#model)
4.	[Persist model](#persistence)
5.	[Deploy and score in a Cloud](#scoring)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup task:

-  Create a [Watson Machine Learning (WML) Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered and information about how to create the instance is [here](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html))

- Configure local python environment
  + python 3.5
  + scikit-learn 0.19.1
  + watson-machine-learning-client

**Tip**: You can install libraries from [PyPI](https://pypi.python.org/pypi) running cell below.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [2]:
!pip install watson-machine-learning-client --upgrade

    100% |████████████████████████████████| 563kB 1.6MB/s eta 0:00:01
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: urllib3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learn

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data from scikit-learn sample datasets and perform a basic exploration.

In [3]:
import sklearn
from sklearn import datasets

digits = datasets.load_digits()

Loaded toy dataset consists of 8x8 pixels images of hand-written digits.

Let's display first digit data and label using **data** and **target**.

In [4]:
print(digits.data[0].reshape((8, 8)))

[[  0.   0.   5.  13.   9.   1.   0.   0.]
 [  0.   0.  13.  15.  10.  15.   5.   0.]
 [  0.   3.  15.   2.   0.  11.   8.   0.]
 [  0.   4.  12.   0.   0.   8.   8.   0.]
 [  0.   5.   8.   0.   0.   9.   8.   0.]
 [  0.   4.  11.   0.   1.  12.   7.   0.]
 [  0.   2.  14.   5.  10.  12.   0.   0.]
 [  0.   0.   6.  13.  10.   0.   0.   0.]]


In [5]:
digits.target[0]

0

In next step you will count data examples.

In [6]:
samples_count = len(digits.images)

print("Number of samples: " + str(samples_count))

Number of samples: 1797


<a id="model"></a>
## 3. Create a Scikit-learn model

In this section you will learn how to prepare data, create a Scikit-learn machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test, and score datasets.

In [7]:
train_data = digits.data[: int(0.7*samples_count)]
train_labels = digits.target[: int(0.7*samples_count)]

test_data = digits.data[int(0.7*samples_count): int(0.9*samples_count)]
test_labels = digits.target[int(0.7*samples_count): int(0.9*samples_count)]

score_data = digits.data[int(0.9*samples_count): ]

print("Number of training records: " + str(len(train_data)))
print("Number of testing records : " + str(len(test_data)))
print("Number of scoring records : " + str(len(score_data)))

Number of training records: 1257
Number of testing records : 360
Number of scoring records : 180


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The score data set will be used for scoring in Cloud.

### 3.2: Create pipeline and train a model

In this section you will create scikit-learn machine learning pipeline and then train the model.

In the first step you need to import the scikit-learn machine learning packages that will be needed in the subsequent steps.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import svm, metrics

Standardize features by removing the mean and scaling to unit variance.

In [9]:
scaler = preprocessing.StandardScaler()

Next, define estimators you want to use for classification. Support Vector Machines (SVM) with radial basis function as kernel is used in the following example.

In [10]:
clf = svm.SVC(kernel='rbf')

Let's build the pipeline now. A pipeline consists of transformer and an estimator.

In [11]:
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])

Now, you can train your SVM model by using the previously defined **pipeline** and **train data**.

In [12]:
model = pipeline.fit(train_data, train_labels)

You can check your **model quality** now. To evaluate the model, use **test data**.

In [13]:
predicted = model.predict(test_data)

print("Evaluation report: \n\n%s" % metrics.classification_report(test_labels, predicted))

Evaluation report: 

             precision    recall  f1-score   support

          0       1.00      0.97      0.99        37
          1       0.97      0.97      0.97        34
          2       1.00      0.97      0.99        36
          3       1.00      0.94      0.97        35
          4       0.78      0.97      0.87        37
          5       0.97      0.97      0.97        38
          6       0.97      0.86      0.91        36
          7       0.92      0.97      0.94        35
          8       0.91      0.89      0.90        35
          9       0.97      0.92      0.94        37

avg / total       0.95      0.94      0.95       360



You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your model in Watson Machine Learning repository by using commont python client.

**Tip**: You can check documentation for watson-machine-learning-client by visiting [documentation webpage](https://wml-api-pyclient.mybluemix.net).

### 4.1: Work with your instance

First, you must import client libraries.

In [14]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Authenticate to Watson Machine Learning service on IBM Cloud.

Authenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the [Service Credentials](https://console.bluemix.net/docs/services/service_credentials.html#service_credentials) tab of the service instance that you created on IBM Cloud. <BR>If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.


In [14]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "****",
  "username": "****",
  "password": "****",
  "instance_id": "****"
}

In [16]:
# The code was removed by DSX for sharing.

#### Create API client by running below code.

In [17]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [19]:
import json

instance_details = client.service_instance.get_details()

### 4.2: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

Define model name, autor name and email.

In [20]:
published_model = client.repository.store_model(model=model, meta_props={'name':'Digits prediction model'}, \
                                                training_data=train_data, training_target=train_labels)

### 4.3: Get model details

In [21]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "modified_at": "2018-03-28T13:00:55.963Z",
    "guid": "ae7ea0bf-3a48-453d-984a-07595ad0ef8a",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/ae7ea0bf-3a48-453d-984a-07595ad0ef8a",
    "created_at": "2018-03-28T13:00:55.885Z"
  },
  "entity": {
    "deployments": {
      "count": 0,
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/ae7ea0bf-3a48-453d-984a-07595ad0ef8a/deployments"
    },
    "input_data_schema": {
      "labels": {
        "fields": [
          {
            "type": "int",
            "name": "l1"
          }
        ],
        "type": "ndarray"
      },
      "features": {
        "fields": [
          {
            "type": "float",
            "name": "f0"
          },
          {
            "type": "float",
            "name": "f1"
          },
          {
            "type": "float",
            "name"

#### Get all models

In [22]:
models_details = client.repository.list_models()

------------------------------------  ----------------------------------  ------------------------  -----------------
GUID                                  NAME                                CREATED                   FRAMEWORK
43986a55-0f0e-493b-9b98-0712f918b0ba  MNIST CNN model                     2018-03-15T16:12:06.503Z  tensorflow-1.5
a5a952e1-fa6f-4c64-88f7-293b367b7e1f  MNIST best model                    2018-03-16T10:12:16.773Z  tensorflow-1.5
c755c740-dab6-4d5d-b873-1060885d55d5  MNIST best model                    2018-03-16T10:12:31.990Z  tensorflow-1.5
861b6b73-6861-49c0-867c-d171d6d9d545  MNIST best model                    2018-03-16T13:00:40.461Z  tensorflow-1.5
f7219c8f-fccb-4d26-a1bd-f51544090eae  training-S3ttnlgig_2 best model     2018-03-16T13:32:33.747Z  tensorflow-1.5
327242a2-d2c0-400f-b673-4f33ad7ec17d  product-line-prediction             2018-03-16T18:08:31.260Z  mllib-2.0
a0cb79ca-e937-4cbf-b7e5-23dad2d2ecec  drug-selection                      2018-03-16T18

### 4.3: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [23]:
loaded_model = client.repository.load(published_model_uid)

You can make predictions to check that model has been loaded correctly.

In [24]:
test_predictions = loaded_model.predict(test_data[:10])

In [25]:
print(test_predictions)

[4 0 5 3 6 9 6 4 7 5]


As you can see you are able to make predictions, so model is loaded back correctly. You have already learned how save to and load the model from Watson Machine Learning repository.

### 4.4: Delete model

You can delete published model from Watson Machine Learning repository using below code. The code is commented out at this stage since the model will be needed later for deployment.

In [27]:
# client.repository.delete(published_model_uid)

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client.

### 5.1: Create model deployment

#### Create online deployment for published model

In [26]:
created_deployment = client.deployments.create(published_model_uid, "Deployment of scikit model")



#######################################################################################

Synchronous deployment creation for uid: '278bff75-ee3d-42c6-a3d1-32dfa51698c1' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='278bff75-ee3d-42c6-a3d1-32dfa51698c1'
------------------------------------------------------------------------------------------------




**Note**: Here we use deployment url saved in published_model object. In next section we show how to retrive deployment url from Watson Mchine Learning instance.

Now you can print an online scoring endpoint. 

In [27]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/ae7ea0bf-3a48-453d-984a-07595ad0ef8a/deployments/278bff75-ee3d-42c6-a3d1-32dfa51698c1/online


### 5.2: Get deployments

In [28]:
deployments = client.deployments.get_details()

You can get deployment_url by parsing deployment details for last deployed model.

In [29]:
deployment_url = client.deployments.get_url(created_deployment)

print(deployment_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/ae7ea0bf-3a48-453d-984a-07595ad0ef8a/deployments/278bff75-ee3d-42c6-a3d1-32dfa51698c1


### 5.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [30]:
scoring_payload = {"values": [list(score_data[0]), list(score_data[1])]}

Use ``client.deployments.score()`` method to run scoring.

In [31]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [32]:
print(json.dumps(predictions, indent=2))

{
  "fields": [
    "prediction"
  ],
  "values": [
    [
      5
    ],
    [
      2
    ]
  ]
}


### 5.4: Delete deployment

Use the following method to delete deployment.

In [33]:
client.deployments.delete(client.deployments.get_uid(created_deployment))

You can check depoyments by running ``list`` method.

In [34]:
client.deployments.list()

------------------------------------  -------------------------------  ------  --------------  ------------------------  --------------
GUID                                  NAME                             TYPE    STATE           CREATED                   FRAMEWORK
0efa6832-80ea-4637-baed-771827562112  MNIST keras deployment           online  DEPLOY_FAILURE  2018-03-15T16:12:48.062Z  tensorflow-1.5
1bf606e8-e8c4-4143-8727-5b2d1ac97b7e  MNIST keras deployment           online  DEPLOY_SUCCESS  2018-03-16T13:00:52.773Z  tensorflow-1.5
2436e614-5396-4b78-ac02-3e6a69479b1c  MNIST keras deployment           online  DEPLOY_FAILURE  2018-03-15T16:13:29.910Z  tensorflow-1.5
4bcae2dc-b309-4e0d-8858-2c04fe4c6383  Mnist model deployment           online  DEPLOY_SUCCESS  2018-03-28T09:02:49.220Z  tensorflow-1.5
4fb26af0-e4b5-4fcc-9156-2fe6dcad41a2  drug deployment                  online  DEPLOY_SUCCESS  2018-03-16T18:11:33.498Z  mllib-2.0
687c8e42-86e8-4b2c-a1bb-c9f2ebebfa23  MNIST keras deployme

### 5.5: Delete model

In [35]:
client.repository.delete(published_model_uid)

You can check your stored models by running below ``list`` method.

In [36]:
client.repository.list_models()

------------------------------------  ----------------------------------  ------------------------  --------------
GUID                                  NAME                                CREATED                   FRAMEWORK
43986a55-0f0e-493b-9b98-0712f918b0ba  MNIST CNN model                     2018-03-15T16:12:06.503Z  tensorflow-1.5
a5a952e1-fa6f-4c64-88f7-293b367b7e1f  MNIST best model                    2018-03-16T10:12:16.773Z  tensorflow-1.5
c755c740-dab6-4d5d-b873-1060885d55d5  MNIST best model                    2018-03-16T10:12:31.990Z  tensorflow-1.5
861b6b73-6861-49c0-867c-d171d6d9d545  MNIST best model                    2018-03-16T13:00:40.461Z  tensorflow-1.5
f7219c8f-fccb-4d26-a1bd-f51544090eae  training-S3ttnlgig_2 best model     2018-03-16T13:32:33.747Z  tensorflow-1.5
327242a2-d2c0-400f-b673-4f33ad7ec17d  product-line-prediction             2018-03-16T18:08:31.260Z  mllib-2.0
a0cb79ca-e937-4cbf-b7e5-23dad2d2ecec  drug-selection                      2018-03-16T18:10

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html)
 for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Wojciech Sobala** is a Data Scientist at IBM developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.